In [1]:
#https://qiita.com/ganyariya/items/25824f1502478a673005


!apt -qq update
!apt -qq -y install libnvtoolsext1
!apt -qq -y install xvfb freeglut3-dev ffmpeg
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay
!pip install -q gym-super-mario-bros==7.3.0
!pip install -q opencv-python

8 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  libnvtoolsext1
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 28.8 kB of archives.
After this operation, 140 kB of additional disk space will be used.
Selecting previously unselected package libnvtoolsext1:amd64.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack .../libnvtoolsext1_11.5.114~11.5.1-1ubuntu1_amd64.deb ...
Unpacking libnvtoolsext1:amd64 (11.5.114~11.5.1-1ubuntu1) ...
Setting up libnvtoolsext1:amd64 (11.5.114~11.5.1-1ubuntu1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.1) ...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev
  libgles1 libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev
  libglx-dev libice-dev libopengl-dev libsm-dev 

In [58]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
#os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
from enum import IntEnum
import matplotlib.pyplot as plt
import sys
import bisect
from functools import cmp_to_key
import time

#動画用
import matplotlib.animation
from IPython.display import HTML
from matplotlib.animation import PillowWriter
import imageio
from google.colab import files

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

In [59]:
# Super Mario environment for OpenAI Gym
import gym_super_mario_bros


# Initialize Super Mario environment
# https://github.com/Kautenja/gym-super-mario-bros
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0")

# Limit the action-space to
#   0. walk right
#   1. jump right
MOVEMENT=[["right","B"], ["right", "A","B"]]
env = JoypadSpace(env, MOVEMENT) #早くクリアするために右ダッシュと右ダッシュジャンプのみ可能とした


env.reset()
state, reward, done, info = env.step(action=0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API whic

In [60]:

class Parameters:
    def __init__(self, num_of_ants, num_of_vertexes, Q, alpha, beta, rou, max_iterations, initial_vertex, tau_min,
                 tau_max, ant_prob_random, not_change,mC_super_not_change,num_of_action, plot_time,STEP_COUNT):
        self.mC_num_of_ants = num_of_ants
        self.mC_num_of_vertexes = num_of_vertexes
        self.mC_Q = Q
        self.mC_alpha = alpha
        self.mC_beta = beta
        self.mC_rou = rou
        self.mC_max_iterations = max_iterations
        self.mC_initial_vertex = initial_vertex
        self.mC_tau_min = tau_min
        self.mC_tau_max = tau_max
        self.mC_ant_prob_random = ant_prob_random
        self.mC_not_change = not_change
        self.mC_super_not_change=mC_super_not_change
        self.mC_num_of_action=num_of_action
        self.mC_STEP_COUNT=STEP_COUNT
        self.mC_Goal=False

class ACOSolver:
    def __init__(self, num_of_ants, num_of_vertexes, Q, alpha, beta, rou, max_iterations, initial_vertex, tau_min,
                  tau_max, ant_prob_random, not_change,mC_super_not_change,num_of_action, plot_time,STEP_COUNT):
          self.mC_parameters = Parameters(num_of_ants, num_of_vertexes, Q, alpha, beta, rou, max_iterations,
                                          initial_vertex, tau_min,
                                          tau_max, ant_prob_random, not_change,mC_super_not_change,num_of_action,plot_time,STEP_COUNT)
          self.m_graph = Graph(self.mC_parameters)
          self.colony=[]
          self.best_path=[]
          self.best_score=0
          self.super_path=[]
          self.super_score=0
          self.best_scoreList=[]
          self.not_change = 0
          self.not_change_judgement=0
          self.m_cnt_super_not_change = 0


    def run_ant(self):
        x=0
        num_of_ants=self.mC_parameters.mC_num_of_ants
        while not self.mC_parameters.mC_Goal and self.best_score<3161:
            self.colony=[]
            if x==0:
              self.first_ant(num_of_ants,x)
            else:
              self.__update_next_pheromones()
              self.__reset_aco()
              self.action_ant(num_of_ants,x)
            self.best_scoreList.append(self.best_score)
            self.calc_next_pheromone(self.colony)

            print("best_score:{}",self.best_score)
            x+=1
            #答えが局所解に陥ったら強制的にフェロモンを初期化する
            if self.m_cnt_super_not_change > self.mC_parameters.mC_super_not_change:
                self.reset_pheromones()
                self.m_cnt_super_not_change = 0
                x=0




    def first_ant(self,num_of_ants,x):
        ant_scores=[]
        colony=[]
        steps=[]
        print("alphabet")
        not_change_judgement=0
        for a in range (num_of_ants):
          ant_path=[]
          breaker=False
          state = env.reset() # 環境リセット
          not_change=0
          self_not_change=0
          now_x_position=0
          while not breaker :
            action=random.choice([0, 1])
            for i in range(self.mC_parameters.mC_STEP_COUNT):#1行動を4フレームにわたって行う
              state, reward, done, info = env.step(action)
              ant_path.append(action)
              if info['flag_get']:    #ゴール判定
                  print(f"<<< Mario get the flag. GOOOOOOOOOOOOOOOOOOOOOAL! >>>")
                  breaker=True
                  self.mC_parameters.mC_Goal=True
                  break

              if done:                #落下判定

                breaker = True
                break

              if now_x_position==info["x_pos"]:
                self_not_change+=1
              else:
                now_x_position=info["x_pos"]
                self_not_change=0

              if self_not_change>self.mC_parameters.mC_not_change:
                not_change_judgement=1
                breaker=True
                break

          ant_scores.append(info["x_pos"])
          colony.append(ant_path)
          steps.append(len(ant_path))
          print("generation: {} ants: {} steps: {} score: {}".format(x + 1, a + 1, len(ant_path), info["x_pos"]))
        colony,ant_scores=sorts(colony,ant_scores,steps)
        print(ant_scores)
        self.super_path,self.super_score=colony[0],ant_scores[0]
        self.judgement_bestpath(self.super_path,self.super_score)
        self.colony=colony
        self.not_change_judgement=not_change_judgement

    # def goal_judgement(self):
    #   if self.goal:

    def judgement_bestpath(self,path,score):
      if self.best_score<score:
        self.best_path=path
        self.best_score=score
        self.m_cnt_super_not_change=0
      else:
        self.m_cnt_super_not_change+=1

    def action_ant(self,num_of_ants,x):
      ant_scores=[]
      colony=[]
      steps=[]
      for a in range (num_of_ants):
        ant_path=[]
        breaker=False
        i=0
        state = env.reset() # 環境リセット
        self_not_change=0
        now_x_position=0
        not_change_judgement=0
        while not breaker:

          random_p = random.uniform(0.0, 0.999999999)
          action=self.select_path(i)
          for i in range(self.mC_parameters.mC_STEP_COUNT):#1行動を4フレームにわたって行う
            state, reward, done, info = env.step(action)
            ant_path.append(action)
            i+=1
            if done:                #落下判定
              breaker = True
              break

            if info['flag_get']:    #ゴール判定
                print(f"<<< Mario get the flag. GOOOOOOOOOOOOOOOOOOOOOAL! >>>")
                breaker=True
                self.mC_parameters.mC_Goal=True
                break
            if now_x_position==info["x_pos"]:
              self_not_change+=1
            else:
              now_x_position=info["x_pos"]
              self_not_change=0

            if self_not_change>self.mC_parameters.mC_not_change:
              not_change_judgement=1
              breaker=True
              break




        ant_scores.append(info["x_pos"])
        colony.append(ant_path)
        steps.append(len(ant_path))
        print("generation: {} ants: {} steps: {} score: {}".format(x + 1, a + 1, len(ant_path), info["x_pos"]))
      colony,ant_scores=sorts(colony,ant_scores,steps)
      print(ant_scores)
      self.super_path,self.super_score=colony[0],ant_scores[0]
      self.judgement_bestpath(self.super_path,self.super_score)
      self.colony=colony
      self.not_change_judgement=not_change_judgement


    def select_path(self,j):
      # もし一様乱数よりも小さいなら、完全ランダムで選ぶ
      to_vertexes=[x for x in range(self.mC_parameters.mC_num_of_action)]
      if random.random() < self.mC_parameters.mC_ant_prob_random:
        to = to_vertexes[random.randint(0, len(to_vertexes) - 1)]
      else:
        to_vertexes, to_prob = self.__calc_prob_from_v(j)
        random_p = random.uniform(0.0, 0.999999999)
        to = to_vertexes[bisect.bisect_left(to_prob, random_p)]
      return to

    def __update_next_pheromones(self):
      num_of_vertexes = self.mC_parameters.mC_num_of_vertexes
      num_of_action=self.mC_parameters.mC_num_of_action
      rou = self.mC_parameters.mC_rou
      tau_min = self.mC_parameters.mC_tau_min
      tau_max = self.mC_parameters.mC_tau_max
      for i in range(num_of_vertexes):
          for j in range(num_of_action):
              self.m_graph.m_edge_pheromone[i][j] = self.m_graph.m_edge_pheromone[i][j] * rou + \
                                                    self.m_graph.m_edge_next_pheromone[i][j]
      # for i in range(num_of_vertexes):
      #     for j in range(num_of_vertexes):
      #         self.m_graph.m_edge_pheromone[i][j] = min(tau_max, max(self.m_graph.m_edge_pheromone[i][j], tau_min))

    def __reset_aco(self):
      self.m_graph.reset_graph()

    def __calc_prob_from_v(self,v):
      # 確率の合計(分母)
      sumV=0
      # 行き先候補情報
      to_vertexes=[x for x in range(self.mC_parameters.mC_num_of_action)]
      to_pheromones=[]
      alpha = self.mC_parameters.mC_alpha*(0.5**self.m_cnt_super_not_change)
      beta = self.mC_parameters.mC_beta**(0.5*self.not_change_judgement)

      for to in range(len(to_vertexes)):
          # フェロモン分子の計算
          pheromone = self.m_graph.m_edge_pheromone[v][to] ** alpha * self.m_graph.m_edge_heuristics[v][to] ** beta
          sumV += pheromone

          to_pheromones.append(pheromone)
      if sumV==0:
        to_pheromones.clear()
        for x in range(len(to_vertexes)):
          pheromone = random.randint(1,30)
          to_pheromones.append(pheromone)
          sumV+=pheromone



      # 分母で割る
      to_prob = [x / sumV for x in to_pheromones]

      # 確率の累積和を取る
      for i in range(len(to_prob) - 1):
          to_prob[i + 1] += to_prob[i]

      # 行き先の確率を返す
      return to_vertexes,to_prob

    def calc_next_pheromone(self,max_path):
      for s,ant_pathes in enumerate(self.colony):
        for k,path in enumerate(ant_pathes):
            t=path
            self.m_graph.m_edge_next_pheromone[k][t]+=(self.mC_parameters.mC_num_of_ants-s)*(0.5**self.m_cnt_super_not_change)

    def reset_pheromones(self):
        self.m_graph.reset_graph_when_stagnation()

In [61]:
class Graph:
  def __init__(self, parameters):
    self.m_num_of_vertexes = parameters.mC_num_of_vertexes
    self.m_num_of_action=parameters.mC_num_of_action
    # self.m_coordinates = [None] * self.m_num_of_vertexes
    # self.m_edge_length = [[0 for j in range(self.m_num_of_vertexes)] for i in range(self.m_num_of_vertexes)]
    self.m_edge_pheromone = [[0 for j in range(self.m_num_of_action)] for i in range(self.m_num_of_vertexes)]
    self.m_edge_heuristics = [[0 for j in range(self.m_num_of_action)] for i in range(self.m_num_of_vertexes)]
    self.m_edge_next_pheromone = [[0 for j in range(self.m_num_of_action)] for i in range(self.m_num_of_vertexes)]
    self.m_parameters = parameters
    self.__prepare_graph()

  def __prepare_graph(self):
    for i in range(self.m_num_of_vertexes):
                for j in range(self.m_num_of_action):
                    self.m_edge_heuristics[i][j] = i

  def reset_graph(self):  #リセットするのは、m_edge_next_pheromoneの次のフェロモン量を計算するための一時配列
    self.m_edge_next_pheromone = [[0 for j in range(self.m_num_of_action)] for i in
                                  range(self.m_num_of_vertexes)]
  def reset_graph_when_stagnation(self):
        self.m_edge_pheromone = [[0 for j in range(self.m_num_of_action)] for i in range(self.m_num_of_vertexes)]




In [62]:
class plot:
  def __init__(self,super_scoreList):
    pass

  def play_plot(self,super_scoreList):
    print(super_scoreList)
    plt.plot(super_scoreList)
    plt.xlabel('世代数')
    plt.ylabel('Awards')
    plt.title('MarioAI by GA')
    plt.show()

In [63]:
#蟻をscore順にソートする関数
#遺伝子をscore順にソートする関数
def sorts(genes, scores,steps):
    initial_list = [{'x': gene, 'y': score, 'z': step} for gene, score, step in zip(genes, scores, steps)]
    sorted_list = sorted(initial_list, key=cmp_to_key(compare))
    #sorted_GENES,sorted_scores,sorted_steps= zip(*sorted_list)
    sorted_scores = [item["y"] for item in sorted_list]
    sorted_GENES = [item["x"] for item in sorted_list]
    return list(sorted_GENES), list(sorted_scores)

def compare(arg1, arg2):
    if arg1["y"] < arg2["y"]:
        return 1
    elif arg1["y"] > arg2["y"]:
        return -1
    else:
        if arg1["z"] < arg2["z"]:
            return -1
        elif arg1["z"] > arg2["z"]:
            return 1
        else:
            return 0

In [64]:

# グラフの初期化
aco = ACOSolver(num_of_ants=10, num_of_vertexes=10000, Q=100, alpha=0.5, beta=0.5, rou=0.5, max_iterations=300,
                initial_vertex=0, tau_min=0, tau_max=3000, ant_prob_random=0.1, not_change=3000,mC_super_not_change=30,
                num_of_action=len(MOVEMENT), plot_time=0.2,STEP_COUNT=4)

# ACOアルゴリズムの実行
aco.run_ant()





alphabet


/usr/local/lib/python3.10/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


generation: 1 ants: 1 steps: 1791 score: 1128
generation: 1 ants: 2 steps: 114 score: 311
generation: 1 ants: 3 steps: 114 score: 315
generation: 1 ants: 4 steps: 124 score: 298
generation: 1 ants: 5 steps: 1486 score: 869
generation: 1 ants: 6 steps: 2211 score: 1417
generation: 1 ants: 7 steps: 822 score: 1414
generation: 1 ants: 8 steps: 764 score: 898
generation: 1 ants: 9 steps: 4223 score: 1128
generation: 1 ants: 10 steps: 1224 score: 820
[1417, 1414, 1128, 1128, 898, 869, 820, 315, 311, 298]
best_score:{} 1417
generation: 2 ants: 1 steps: 112 score: 311
generation: 2 ants: 2 steps: 282 score: 702
generation: 2 ants: 3 steps: 354 score: 691
generation: 2 ants: 4 steps: 320 score: 693
generation: 2 ants: 5 steps: 590 score: 898
generation: 2 ants: 6 steps: 112 score: 314
generation: 2 ants: 7 steps: 606 score: 857
generation: 2 ants: 8 steps: 288 score: 700
generation: 2 ants: 9 steps: 898 score: 804
generation: 2 ants: 10 steps: 274 score: 680
[898, 857, 804, 702, 700, 693, 691,

In [3]:
plot=aco.plot(aco.best_scoreList)
aco.plot.play_plot(aco.best_scoreList)

frames=[]

state = env.reset() # 環境リセット
for i, move in enumerate(aco.best_path):

  state, reward, done, info = env.step(move)
  frames.append(copy.deepcopy(env.render(mode='rgb_array')))
print( 'reward:', aco.best_score, 'step:', len(aco.best_path))
SKIP_RATE = 2
if SKIP_RATE == 1:
  frames_new = np.array(frames)
else:
  np_frames = np.array(frames)
  frames_new = np_frames[::SKIP_RATE].tolist()

env.close()

NameError: ignored

In [ ]:

# アニメーションのサイズと解像度を調整
fig, ax = plt.subplots(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
ax.axis('off')

# imshow()を使って描画
patch = ax.imshow(frames[0])
# アニメーションの更新関数
def animate(i):
    patch.set_data(frames[i])


# FuncAnimationを作成
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(frames), interval=50)

# アニメーションをGIFファイルとして保存
writer = PillowWriter(fps=30)
ani.save('animation.gif', writer=writer)

# アニメーションを表示
HTML(ani.to_jshtml())

matplotlib.rcParams['animation.embed_limit'] = 2**128
plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
patch = plt.imshow(frames_new[0])
plt.axis('off')
animate = lambda i: patch.set_data(frames_new[i])
ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames_new), interval = 50)
HTML(ani.to_jshtml())




In [ ]:
SKIP_RATE=2
imageio.mimsave('ai_mario.gif', [np.array(img) for i, img in enumerate(frames) if i%SKIP_RATE == 0], fps=29)
files.download('ai_mario.gif')